<a href="https://colab.research.google.com/github/thkim-us/google-drive-migration-helper/blob/main/notebooks/migration_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive 마이그레이션 테스트 (간단 버전)

Google Colab의 내장 인증을 사용합니다.

## 사용 방법
1. 셀을 순서대로 실행하세요
2. Google 계정으로 로그인하세요
3. 파일 목록을 확인하세요


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# ============================================
# Google Drive 폴더 URL → 내 드라이브로 재귀 복사
# - 진행률(progress bar) + 상세 로그 출력 + 실패 시 계속 진행
# ============================================

# 1) 공유 폴더 URL을 그대로 붙여넣으세요.
SRC_FOLDER_URL = "https://drive.google.com/drive/folders/1jpeF6TwCz8ATAUjrge57fdyLaPmw8hzB"

# 2) 내 드라이브에 생성할 상위 대상 폴더명
DEST_FOLDER_NAME = "개인 드라이브 백업"

# 3) 옵션: 로그 파일을 MyDrive에 저장할지 여부
SAVE_LOGS_TO_MYDRIVE = True

# --------------------------------------------
# 준비: 인증 + 라이브러리
# --------------------------------------------
from google.colab import auth
auth.authenticate_user()

!pip -q install tqdm
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from tqdm import tqdm
import time, os, csv, datetime

service = build("drive", "v3")

# --------------------------------------------
# 유틸: URL → Folder ID 추출
# --------------------------------------------
def extract_folder_id(url: str) -> str:
    if "folders/" in url:
        return url.split("folders/")[1].split("?")[0].strip().strip("/")
    raise ValueError("올바른 구글 드라이브 폴더 URL을 입력하세요. (예: .../drive/folders/<FOLDER_ID>)")

SRC_FOLDER_ID = extract_folder_id(SRC_FOLDER_URL)
print("📂 추출된 Folder ID:", SRC_FOLDER_ID)

# --------------------------------------------
# 유틸: 대상 폴더 확인/생성
# --------------------------------------------
def ensure_folder(name: str, parent_id: str = "root") -> str:
    q = (
        f"trashed=false and mimeType='application/vnd.google-apps.folder' "
        f"and name='{name.replace('\"','\\\"')}' and '{parent_id}' in parents"
    )
    res = service.files().list(
        q=q,
        fields="files(id,name)",
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        corpora="allDrives",
        pageSize=1,
    ).execute()
    files = res.get("files", [])
    if files:
        return files[0]["id"]

    folder_metadata = {
        "name": name,
        "mimeType": "application/vnd.google-apps.folder",
        "parents": [parent_id],
    }
    new_folder = service.files().create(
        body=folder_metadata,
        fields="id",
        supportsAllDrives=True,
    ).execute()
    return new_folder["id"]

# --------------------------------------------
# 유틸: 폴더 내 목록 (paging)
# --------------------------------------------
def list_children(folder_id: str):
    page_token = None
    while True:
        res = service.files().list(
            q=f"trashed=false and '{folder_id}' in parents",
            spaces="drive",
            fields="nextPageToken, files(id, name, mimeType)",
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            corpora="allDrives",
            pageToken=page_token,
            pageSize=1000,
        ).execute()
        for item in res.get("files", []):
            yield item
        page_token = res.get("nextPageToken", None)
        if not page_token:
            break

# --------------------------------------------
# 유틸: 재귀적으로 트리 탐색 (경로 포함)
# --------------------------------------------
GOOGLE_FOLDER = "application/vnd.google-apps.folder"

def walk_tree(folder_id: str, base_path: str=""):
    """yield dict: {id, name, mimeType, path} (파일/폴더 모두)"""
    for item in list_children(folder_id):
        path = f"{base_path}/{item['name']}".lstrip("/")
        yield {"id": item["id"], "name": item["name"], "mimeType": item["mimeType"], "path": path}
        if item["mimeType"] == GOOGLE_FOLDER:
            yield from walk_tree(item["id"], path)

# --------------------------------------------
# 유틸: 파일 복사 (재시도 포함)
# --------------------------------------------
def copy_file(file_id: str, new_name: str, parent_id: str):
    body = {"name": new_name, "parents": [parent_id]}
    for attempt in range(6):  # 최대 6회(backoff 1,2,4,8,16초)
        try:
            return service.files().copy(
                fileId=file_id,
                body=body,
                fields="id",
                supportsAllDrives=True,
            ).execute()
        except HttpError as e:
            # 일시적/쿼터/속도 제한 에러 → 백오프 후 재시도
            if e.resp.status in (403, 429, 500, 503):
                sleep_s = 2 ** attempt
                time.sleep(sleep_s)
                continue
            raise

# --------------------------------------------
# 유틸: 폴더 복사 (재귀) + 진행률/로그
# --------------------------------------------
def copy_folder_with_progress(src_folder_id: str, dst_parent_id: str, dst_name: str):
    # 1) 전체 파일 수 사전 스캔
    print("📊 사전 스캔 중... (총 파일 수 집계)")
    all_nodes = list(walk_tree(src_folder_id))
    total_files = sum(1 for n in all_nodes if n["mimeType"] != GOOGLE_FOLDER)
    print(f"총 파일 수: {total_files:,} (폴더 제외)\n")

    # 2) 대상 루트 폴더 확보
    dst_root_id = ensure_folder(dst_name, dst_parent_id)
    dst_root_url = f"https://drive.google.com/drive/folders/{dst_root_id}"

    # 3) 경로 → 대상 폴더ID 캐시
    folder_cache = {"": dst_root_id}

    # 4) 로그 준비
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_rows = []
    log_header = ["timestamp","type","src_id","src_path","dst_id","dst_path","status","message"]
    log_dir = "/content/drive/MyDrive/copy_logs" if SAVE_LOGS_TO_MYDRIVE else "/content"
    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, f"drive_copy_log_{ts}.csv")

    def log_event(row):
        log_rows.append(row)
        with open(log_path, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            if f.tell() == 0:
                w.writerow(log_header)
            w.writerow(row)

    # 5) 진행바
    pbar = tqdm(total=total_files, unit="file", desc="복사 진행", ncols=100)

    # 6) 트리 순회하며 복사
    for node in all_nodes:
        ntype = "folder" if node["mimeType"] == GOOGLE_FOLDER else "file"
        now = datetime.datetime.now().isoformat(timespec="seconds")

        if ntype == "folder":
            parent_path = "/".join(node["path"].split("/")[:-1])
            parent_id = folder_cache.get(parent_path, dst_root_id)
            if node["path"] not in folder_cache:
                new_id = ensure_folder(node["name"], parent_id)
                folder_cache[node["path"]] = new_id
                log_event([now,"folder",node["id"],node["path"],new_id,node["path"],"OK","folder ensured"])
        else:
            parent_path = "/".join(node["path"].split("/")[:-1])
            parent_id = folder_cache.get(parent_path, dst_root_id)
            try:
                copied = copy_file(node["id"], node["name"], parent_id)
                log_event([now,"file",node["id"],node["path"],copied["id"],node["path"],"OK","copied"])
            except Exception as e:
                log_event([now,"file",node["id"],node["path"],"",node["path"],"FAIL",str(e)])
            finally:
                pbar.update(1)
                pbar.set_postfix_str(node["path"][:50])

    pbar.close()
    print(f"\n🧾 로그 저장 위치: {log_path}")
    print(f"📂 최종 대상 폴더: {dst_root_url}")  # 👈 링크 출력
    return dst_root_id, dst_root_url

# 실행
print(f"🚀 복사 시작: SRC({SRC_FOLDER_ID}) → 내 드라이브 / '{DEST_FOLDER_NAME}'")
dst_id, dst_url = copy_folder_with_progress(SRC_FOLDER_ID, "root", DEST_FOLDER_NAME)

📂 추출된 Folder ID: 1jpeF6TwCz8ATAUjrge57fdyLaPmw8hzB
🚀 복사 시작: SRC(1jpeF6TwCz8ATAUjrge57fdyLaPmw8hzB) → 내 드라이브 / '개인 드라이브 백업'
📊 사전 스캔 중... (총 파일 수 집계)
총 파일 수: 4 (폴더 제외)



복사 진행: 100%|█| 4/4 [00:12<00:00,  3.21s/file, worksync project/Project_WorkSync_발표자료_ᄎ


🧾 로그 저장 위치: /content/drive/MyDrive/copy_logs/drive_copy_log_20250908_080542.csv
📂 최종 대상 폴더: https://drive.google.com/drive/folders/1fFTWTIwjC3UVQVJoVCA8utImgaOHv8Ta
✅ 완료! 대상 폴더 링크: https://drive.google.com/drive/folders/1fFTWTIwjC3UVQVJoVCA8utImgaOHv8Ta
